# Visualizing Filters of a Convolutional Neural Network

## Task 2: Downloading the Model

In [ ]:
%matplotlib inline

import tensorflow as tf
import matplotlib.pyplot as plt
import random

print('Using TensorFlow', tf.__version__)

In [ ]:
model = tf.keras.applications.vgg16.VGG16(
    include_top=False,
    weights='imagenet',
    input_shape=(96, 96, 3)
)

model.summary()

## Task 3: Get Layer Output

In [ ]:
def get_submodel(layer_name):
    return tf.keras.models.Model(
        model.input,
        model.get_layer(layer_name).output
    )

get_submodel('block1_conv2').summary()

## Task 4: Image Visualization

In [ ]:
def plot_image(image, title='random image'):
    image = image - tf.math.reduce_min(image)
    image = image / tf.math.reduce_max(image)
    plt.imshow(image)
    plt.title(title)
    plt.xticks([])
    plt.yticks([])
    plt.show()

In [ ]:
image = tf.random.uniform((96, 96, 3), minval=-0.5, maxval=0.5)
plot_image(image)

## Task 5: Training Loop

In [ ]:
def visualize_filter(layer_name, lr=20, iters=50, f_index=None):
    submodel = get_submodel(layer_name)
    
    num_filters = submodel.output.shape[-1]

    if not f_index:
        f_index = random.randint(0, num_filters - 1)
    
    assert num_filters > f_index, f'f_index ({f_index}) >= num_filters ({num_filters})'

    print(f'Maximizing image for layer {layer_name} and filter {f_index}')

    image = tf.random.uniform((96, 96, 3), minval=-0.5, maxval=0.5)

    for i in range(0, iters):
        with tf.GradientTape() as tape:
            tape.watch(image)
            output = submodel(tf.expand_dims(image, axis=0))[:,:,:,f_index]
            loss = tf.math.reduce_mean(output)
        grads = tape.gradient(loss, image)
        grads = tf.math.l2_normalize(grads)
        image += grads * lr

    plot_image(image, f'{layer_name} ({f_index})')

## Task 6: Final Results

In [ ]:
print([layer.name for layer in model.layers])

In [ ]:
layer_name = 'block4_conv2' #@param []
visualize_filter(layer_name=layer_name)